Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/get-started-day1/day1-part4-data.png)

# Tutorial: Bring your own data (Part 4 of 4)

---
## Introduction

In the previous [Tutorial: Train a model in the cloud](day1-part3-train-model.ipynb) article, the CIFAR10 data was downloaded using the inbuilt `torchvision.datasets.CIFAR10` method in the PyTorch API. However, in many cases you are going to want to use your own data in a remote training run. This article focuses on the workflow you can leverage such that you can work with your own data in Azure Machine Learning. 

By the end of this tutorial you would have a better understanding of:

- How to upload your data to Azure
- Best practices for working with cloud data in Azure Machine Learning
- Working with command-line arguments

This notebook follows the steps provided on the [Python (day 1) - bring your own data documentation page](https://aka.ms/day1aml).

## Prerequisites

- You have completed:
  - Setup on your [Azure Machine Learning Compute Cluster](day1-part1-setup.ipynb)
  - [Tutorial: Hello World](day1-part2-hello-world.ipynb)
  - [Tutorial: Train a model in the cloud](day1-part3-train-model.ipynb)
- Familiarity with Python and Machine Learning concepts
- If you are using a compute instance in Azure Machine Learning to run this notebook series, you are all set. Otherwise, please follow the [Configure a development environment for Azure Machine Learning](https://docs.microsoft.com/azure/machine-learning/how-to-configure-environment)

---

## Your machine learning code

By now you have your training script running in Azure Machine Learning, and can monitor the model performance. Let's _parametrize_ the training script by introducing
arguments. Using arguments will allow you to easily compare different hyperparmeters.

Presently our training script is set to download the CIFAR10 dataset on each run. The python code in [code/pytorch-cifar10-your-data/train.py](code/pytorch-cifar10-your-data/train.py) now uses **`argparse` to parametize the script.**

### Understanding your machine learning code changes

The code used in `train.py` has leveraged the `argparse` library to set up the `data_path`, `learning_rate`, and `momentum`.

```python
# .... other code
parser = argparse.ArgumentParser()
parser.add_argument('--data_path', type=str, help='Path to the training data')
parser.add_argument('--learning_rate', type=float, default=0.001, help='Learning rate for SGD')
parser.add_argument('--momentum', type=float, default=0.9, help='Momentum for SGD')
args = parser.parse_args()
# ... other code
```

Also the `train.py` script was adapted to update the optimizer to use the user-defined parameters:

```python
optimizer = optim.SGD(
    net.parameters(),
    lr=args.learning_rate,     # get learning rate from command-line argument
    momentum=args.momentum,    # get momentum from command-line argument
)
```

## Test your machine learning code locally

To run the modified training script locally, run the python command below.

You avoid having to download the CIFAR10 dataset by passing in a local path to the
data. Also you can experiment with different values for _learning rate_ and
_momentum_ hyperparameters without having to hard-code them in the training script.


In [ ]:
!python code/pytorch-cifar10-your-data/train.py --data_path ./data --learning_rate 0.003 --momentum 0.92

## Upload your data to Azure

In order to run this script in Azure Machine Learning, you need to make your training data available in Azure. Your Azure Machine Learning workspace comes equipped with a _default_ **Datastore** - an Azure Blob storage account - that you can use to store your training data.

> <span style="color:purple; font-weight:bold">! NOTE <br>
> Azure Machine Learning allows you to connect other cloud-based datastores that store your data. For more details, see [datastores documentation](./concept-data.md).</span>


In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./data', target_path='datasets/cifar10', overwrite=True)

The `target_path` specifies the path on the datastore where the CIFAR10 data will be uploaded.

## Submit your machine learning code to Azure Machine Learning

As you have done previously, create a new Python control script:

In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

ws = Workspace.from_config()

datastore = ws.get_default_datastore()
dataset = Dataset.File.from_files(path=(datastore, 'datasets/cifar10'))

experiment = Experiment(workspace=ws, name='day1-experiment-data')

config = ScriptRunConfig(source_directory='./code/pytorch-cifar10-your-data',
                         script='train.py',
                         compute_target='cpu-cluster',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_mount(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])

# set up pytorch environment
env = Environment.from_conda_specification(name='pytorch-aml-env',file_path='configuration/pytorch-aml-env.yml')
config.run_config.environment = env

run = experiment.submit(config)
RunDetails(run).show()

### Understand the control code

The above control code has the following additional code compared to the control code written in [previous tutorial](03-train-model.ipynb)

**`dataset = Dataset.File.from_files(path=(datastore, 'datasets/cifar10'))`**: A Dataset is used to reference the data you uploaded to the Azure Blob Store. Datasets are an abstraction layer on top of your data that are designed to improve reliability and trustworthiness.


**`config = ScriptRunConfig(...)`**: We modified the `ScriptRunConfig` to include a list of arguments that will be passed into `train.py`. We also specified `dataset.as_named_input('input').as_mount()`, which means the directory specified will be _mounted_ to the compute target.

## Inspect the 70_driver_log log file

In the navigate to the 70_driver_log.txt file - you should see the following output:

```
Processing 'input'.
Processing dataset FileDataset
{
  "source": [
    "('workspaceblobstore', 'datasets/cifar10')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "XXXXX",
    "name": null,
    "version": null,
    "workspace": "Workspace.create(name='XXXX', subscription_id='XXXX', resource_group='X')"
  }
}
Mounting input to /tmp/tmp9kituvp3.
Mounted input to /tmp/tmp9kituvp3 as folder.
Exit __enter__ of DatasetContextManager
Entering Run History Context Manager.
Current directory:  /mnt/batch/tasks/shared/LS_root/jobs/dsvm-aml/azureml/tutorial-session-3_1600171983_763c5381/mounts/workspaceblobstore/azureml/tutorial-session-3_1600171983_763c5381
Preparing to call script [ train.py ] with arguments: ['--data_path', '$input', '--learning_rate', '0.003', '--momentum', '0.92']
After variable expansion, calling script [ train.py ] with arguments: ['--data_path', '/tmp/tmp9kituvp3', '--learning_rate', '0.003', '--momentum', '0.92']

Script type = None
===== DATA =====
DATA PATH: /tmp/tmp9kituvp3
LIST FILES IN DATA PATH...
['cifar-10-batches-py', 'cifar-10-python.tar.gz']
```

Notice:

1. Azure Machine Learning has mounted the blob store to the compute cluster automatically for you.
2. The ``dataset.as_named_input('input').as_mount()`` used in the control script resolves to the mount point
3. In the machine learning code we include a line to list the directorys under the data directory - you can see the list above.

## Clean up resources

The compute cluster will scale down to zero after 40minutes of idle time. When the compute is idle you will not be charged. If you want to delete the cluster use:


In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ct = ws.compute_targets['cpu-cluster']
# ct.delete()

If you're not going to use what you've created here, delete the resources you just created with this quickstart so you don't incur any charges for storage. In the Azure portal, select and delete your resource group.

## Next Steps

To learn more about the capabilities of Azure Machine Learning please refer to the following documentation:

* [Azure Machine Learning Pipelines](https://docs.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines#building-pipelines-with-the-python-sdk)
* [Deploy models for real-time scoring](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-deploy-models-with-aml)
* [Hyper parameter tuning with Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters)
* [Prep your code for production](https://docs.microsoft.com/azure/machine-learning/tutorial-convert-ml-experiment-to-production)